### Функции и создание класса

#### Задание 1
Имеется класс Rate, который делает запрос к сервису https://www.cbr-xml-daily.ru/daily_json.js, обрабатывает словарь от сервиса и оставлять нужную информацию, возвращает инфо для заданной валюты.<br>
Напишите метод, который возвращает название валюты (поле ‘Name’) с максимальным значением курса.<br>
Предусмотрите параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [1]:
class Rate:
    def __init__(self, format_='value'):         
        '''Задает формат ответа: full - вся инфо о валюте, value - только курс.'''        
        self.format = format_
    
    def exchange_rates(self):
        '''
        Посылает запрос к сервису с информацией о валютах https://www.cbr-xml-daily.ru/daily_json.js
        и возвращает ответ в виде:
            {
                'AMD': {
                    'CharCode': 'AMD',
                    'ID': 'R01060',
                    'Name': 'Армянских драмов',
                    'Nominal': 100,
                    'NumCode': '051',
                    'Previous': 14.103,
                    'Value': 14.0879
                    },
                ... }
        '''
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js') 
        return self.r.json()['Valute']
    
    def info(self, currency):
        '''
        Возвращает информацию о валюте currency в двух вариантах:
            * полная информация о валюте при self.format = 'full':
                Rate('full').info('EUR')
                {
                    'CharCode': 'EUR',
                    'ID': 'R01239',
                    'Name': 'Евро',
                    'Nominal': 1,
                    'NumCode': '978',
                    'Previous': 79.6765,
                    'Value': 79.4966        }
            * информация только о курсе при self.format = 'value':
                Rate('value').info('EUR')
                79.4966
        '''
        response = self.exchange_rates()        
        if currency in response:             
            if self.format == 'full':
                return response[currency]            
            if self.format == 'value':
                return response[currency]['Value']        
        return 'Error'
    
    def eur(self):
        '''Возвращает курс евро на сегодня в формате self.format'''
        return self.info('EUR')
    
    def usd(self):
        '''Возвращает курс доллара на сегодня в формате self.format'''
        return self.info('USD')
    
    def brl(self):
        '''Возвращает курс бразильского реала на сегодня в формате self.format'''
        return self.info('BRL')

In [2]:
import requests

Дополняем / вносим изменения в класс Rate:

In [3]:
class Rate:
    
# добавляем параметр diff:
    def __init__(self, format_='value', diff_= False):
        '''
       Задает формат ответа:
           * format : full - вся инфо о валюте, value - только курс.
           * diff : True - изменение курса к прошлому значению, False - текущий курс
        '''  
        self.format = format_ 
        self.diff = diff_        
    
    def exchange_rates(self):
        '''
        Посылает запрос к сервису с информацией о валютах https://www.cbr-xml-daily.ru/daily_json.js
        и возвращает ответ в виде словаря.
        '''
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js') 
        return self.r.json()['Valute']

# вносим изменения в связи с появлением параметра diff:
    def info(self, currency):
        '''
         Возвращает информацию о валюте currency в двух вариантах:
            * полная информация о валюте при self.format = 'full'
            * информация только о курсе при self.format = 'value' и self.diff = False
            * информация о изменении курса к прошлому значению при self.format = 'value' и self.diff = True
            (округление до 4 знаков)
        '''
        response = self.exchange_rates()        
        if currency in response:             
            if self.format == 'full':
                if self.diff:
                    return "Параметр diff рассматривается только при format = 'value'"
                else:
                    return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return round(response[currency]['Value'] - response[currency]['Previous'],4)
                else:
                    return response[currency]['Value']        
        return 'Error'
    
    def eur(self):
        '''Возвращает курс евро на сегодня в формате self.format и self.diff'''
        return self.info('EUR')
    
    def usd(self):
        '''Возвращает курс доллара на сегодня в формате self.format и self.diff'''
        return self.info('USD')
    
    def brl(self):
        '''Возвращает курс бразильского реала на сегодня в формате self.format и self.diff'''
        return self.info('BRL')

# добавляем новый метод:
    def max_rate(self):
        '''
        Возвращает название валюты (поле ‘Name’) с максимальным значением курса (вариант 1).
        Перебирает курсы валют в словаре-ответе на запрос 
        и записывает наибольший курс в max_val и название этой валюты в max_name.
        '''
        max_val = 0
        max_name = ''
        response = self.exchange_rates()
        for value in response.values():       
                if value['Value'] > max_val :
                    max_val = value['Value']
                    max_name = value['Name']
        return max_name
    
# добавляем новый метод:    
    def max_rate_2(self):                                      
        '''
        Возвращает название валюты (поле ‘Name’) с максимальным значением курса (вариант 2).
        Перебирает курсы валют в словаре-ответе на запрос, находит максимальный
        и определяет соответствующее ему название валюты.
        '''
        response = self.exchange_rates()
        return max(response.items(), key = lambda x: x[1]['Value'])[1]['Name']
    
# добавляем новый метод:   
    def max_rate_sdr(self, incl_sdr_ = True):
        '''
        Возвращает название валюты (поле ‘Name’) с максимальным значением курса,
        в качестве параметра incl_sdr можно задать, учитывать курс СДР (incl_sdr = True) или нет.
        Перебирает курсы валют в словаре-ответе на запрос 
        и записывает наибольший курс в max_val и название этой валюты в max_name.
        '''        
        incl_sdr = incl_sdr_
        max_val = 0
        max_name = ''
        response = self.exchange_rates()
        
        if incl_sdr:
            for value in response.values():       
                    if value['Value'] > max_val :
                        max_val = value['Value']
                        max_name = value['Name']
        else:
            for value in response.values():       
                    if value['Value'] > max_val and value['CharCode'] != 'XDR':
                        max_val = value['Value']
                        max_name = value['Name']                  
        return max_name

In [4]:
r_full = Rate(format_='full')
r_value = Rate(format_='value')

In [5]:
r_full.info('EUR')

{'ID': 'R01239',
 'NumCode': '978',
 'CharCode': 'EUR',
 'Nominal': 1,
 'Name': 'Евро',
 'Value': 86.6419,
 'Previous': 88.468}

In [6]:
r_value.info('USD')

77.8174

In [7]:
r_full.eur()

{'ID': 'R01239',
 'NumCode': '978',
 'CharCode': 'EUR',
 'Nominal': 1,
 'Name': 'Евро',
 'Value': 86.6419,
 'Previous': 88.468}

In [8]:
r_value.usd()

77.8174

In [9]:
print(r_full.max_rate())
print(r_full.max_rate_2())
print(r_full.max_rate_sdr())
print(r_full.max_rate_sdr(incl_sdr_ = False))

СДР (специальные права заимствования)
СДР (специальные права заимствования)
СДР (специальные права заимствования)
Фунт стерлингов Соединенного королевства


In [10]:
print(r_value.max_rate())
print(r_value.max_rate_2())
print(r_value.max_rate_sdr())
print(r_value.max_rate_sdr(incl_sdr_ = False))

СДР (специальные права заимствования)
СДР (специальные права заимствования)
СДР (специальные права заимствования)
Фунт стерлингов Соединенного королевства


In [11]:
r = Rate(diff_=True)
print(r.eur())
print(r.info('EUR'))

-1.8261
-1.8261


In [12]:
r = Rate(format_='full',diff_=True)
print(r.usd())

Параметр diff рассматривается только при format = 'value'


In [13]:
r = Rate(format_='full')
print(r.info('USD'))

{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 77.8174, 'Previous': 78.947}


In [14]:
r = Rate(format_='value')
print(r.info('BRL'))
print(r.brl())

14.392
14.392


Альтернативный вариант класса, в котором задавать значение параметров format и diff нужно не при создании инстанса класса, а при вызове методов:

In [15]:
class Rate_2: 
    
    def exchange_rates(self):
        '''
        Посылает запрос к сервису с информацией о валютах https://www.cbr-xml-daily.ru/daily_json.js
        и возвращает ответ в виде словаря.
        '''
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js') 
        return self.r.json()['Valute']
    
    def info(self, currency,  format_ = 'value', diff_ = False):
        '''
         Возвращает информацию о валюте currency в двух вариантах:
            * полная информация о валюте при self.format = 'full'
            * информация только о курсе при self.format = 'value' и self.diff = False
            * информация о изменении курса к прошлому значению при self.format = 'value' и self.diff = True
            (округление до 4 знаков)
        '''
        self.format = format_
        self.diff = diff_         
        response = self.exchange_rates()        
        if currency in response:             
            if self.format == 'full':
                if self.diff:
                    return "Параметр diff рассматривается только при format = 'value'"
                else:
                    return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return round(response[currency]['Value'] - response[currency]['Previous'],4)
                else:
                    return response[currency]['Value']        
        return 'Error'
       
    def eur(self, format_='value', diff_= False):
        '''Возвращает курс евро на сегодня в формате format и diff'''
        return self.info('EUR', format_, diff_)
    
    def usd(self, format_='value', diff_= False):
        '''Возвращает курс доллара на сегодня в формате format и diff'''
        return self.info('USD', format_, diff_)
    
    def brl(self, format_='value', diff_= False):
        '''Возвращает курс бразильского реала на сегодня в формате format и diff'''
        return self.info('BRL', format_, diff_)

In [16]:
r_2 = Rate_2()
print(r_2.eur(format_='value',diff_=True))
print(r_2.info('EUR'))
print(r_2.eur())
print(r_2.info('EUR',format_='full',diff_=True))
print(r_2.info('EUR',format_='full'))

-1.8261
86.6419
86.6419
Параметр diff рассматривается только при format = 'value'
{'ID': 'R01239', 'NumCode': '978', 'CharCode': 'EUR', 'Nominal': 1, 'Name': 'Евро', 'Value': 86.6419, 'Previous': 88.468}


#### Задание 2
Имеется класс Employee, который повышает уровень сотрудника, публикует грейд, публикует решение по результатам аккредитации.
На его основе есть класс Developer для сотрудников соответствующего отдела.<br>
Напишите класс Designer, который учитывает количество международных премий. Повышение на 1 грейд за каждые 7 баллов, получение международной премии – это +2 балла. Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем.

In [17]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority        
        self.grade = 1
    
    def grade_up(self):
        """Повышает грейд сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация грейда сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        """Публикует грейд сотрудника по результатам аккредитации"""
        pass

In [18]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        """Публикует грейд сотрудника по результатам аккредитации.
        Для каждой аккредитации увеличиваем кол-во баллов (seniority) на 1.
        Пока считаем, что все разработчики проходят аккредитацию.
        Повышение на 1 грейд происходит каждые 5 баллов.
        """
        self.seniority += 1
        if self.seniority % 5 == 0:
            self.grade_up()
        return self.publish_grade()

Например, разработчик Александр только пришел в компанию и у него первый грейд:

In [19]:
alex = Developer('Александр', 0)
alex.publish_grade()

Александр 1


По какой-то причине Александру повысили грейд и теперь у него грейд 2:

In [20]:
alex.grade_up()
alex.publish_grade()

Александр 2


Дальше Александр проходил аккредитации и за каждые пять баллов (то есть за каждую пятую аккредитацию) получал повышение грейда:

In [21]:
for i in range(10):
    print(f"Аккредитация №{i+1}: ")
    alex.check_if_it_is_time_for_upgrade()

Аккредитация №1: 
Александр 2
Аккредитация №2: 
Александр 2
Аккредитация №3: 
Александр 2
Аккредитация №4: 
Александр 2
Аккредитация №5: 
Александр 3
Аккредитация №6: 
Александр 3
Аккредитация №7: 
Александр 3
Аккредитация №8: 
Александр 3
Аккредитация №9: 
Александр 3
Аккредитация №10: 
Александр 4


Создаем класс для дизайнеров:

In [22]:
class Designer(Employee):
    
    def __init__(self, name, seniority, int_prize_ = 2):
        self.int_prize = int_prize_  # кол-во международных премий               
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        """Публикует грейд сотрудника по результатам аккредитации.
        Для каждой аккредитации увеличиваем кол-во баллов (seniority) на 1.
        Пока считаем, что все дизайнеры проходят аккредитацию.
        Повышение на 1 грейд происходит каждые 7 баллов.
        Наличие межд. премий учитывается в баллах.
        """
        self.seniority += 1        
        if (self.seniority + self.int_prize * 2) % 7 == 0: # каждая межд. премия стоит 2 балла 
            self.grade_up()
        return self.publish_grade()   

Дизайнер Мария только пришла в компанию и у нее первый грейд:

In [23]:
maria = Designer('Мария',0)
maria.publish_grade()

Мария 1


Дальше Мария проходила аккредитации и за каждые 7 баллов получала повышение грейда. Первое повышение случилось уже после третьей аккредитации, так как благодаря межд. премиям у Марии было сразу 4 балла. А за первую, вторую и третью аккредитации набралось еще три балла:

In [24]:
for i in range(12):
    print(f"Аккредитация №{i+1}: ")
    maria.check_if_it_is_time_for_upgrade()

Аккредитация №1: 
Мария 1
Аккредитация №2: 
Мария 1
Аккредитация №3: 
Мария 2
Аккредитация №4: 
Мария 2
Аккредитация №5: 
Мария 2
Аккредитация №6: 
Мария 2
Аккредитация №7: 
Мария 2
Аккредитация №8: 
Мария 2
Аккредитация №9: 
Мария 2
Аккредитация №10: 
Мария 3
Аккредитация №11: 
Мария 3
Аккредитация №12: 
Мария 3


Если бы Мария пришла в компанию не с двуми, а тремя межд. премиями, то первое повышение грейдов случилось бы уже после первой аккредитации:

In [25]:
maria = Designer('Мария',0,3)
maria.publish_grade()

for i in range(12):
    print(f"Аккредитация №{i+1}: ")
    maria.check_if_it_is_time_for_upgrade()

Мария 1
Аккредитация №1: 
Мария 2
Аккредитация №2: 
Мария 2
Аккредитация №3: 
Мария 2
Аккредитация №4: 
Мария 2
Аккредитация №5: 
Мария 2
Аккредитация №6: 
Мария 2
Аккредитация №7: 
Мария 2
Аккредитация №8: 
Мария 3
Аккредитация №9: 
Мария 3
Аккредитация №10: 
Мария 3
Аккредитация №11: 
Мария 3
Аккредитация №12: 
Мария 3
